[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_add_appearance_signature_field.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Sign a PDF and apply a visual signature appearance
Sign a PDF document using a provided certificate and
apply a visual signature appearance. This process
requires an input PDF that already contains a signature
field. The provided certificate is used to sign the
document and attach the signature to the existing field.
The visual appearance of the signature is updated using
an XML or JSON file, allowing the addition of text,
images, or PDFs. This signature consists of both a
visible and a non-visible part. Only the non-visible part
is used by other applications to verify the integrity of
the signed part of the document and validate the signing
certificate. The signing certificate is retrieved from a
password-protected PKCS#12 file (.pfx or .p12).

In [ ]:
import io
import os
from pdftools_sdk.pdf import Document
from pdftools_sdk.sign import Signer
from pdftools_sdk.crypto.providers.built_in import Provider
from pdftools_sdk.sign.appearance import Appearance

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
certificate_file = 'INSERT-PFX-FILE'  # Placeholder for PFX file
password = 'INSERT-PASSWORD'
appearance_config_file_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/visual_appearance.xml'
appearance_config_file = 'visual_appearance.xml'
prepare_file(appearance_config_file_url, appearance_config_file)
appearance_file_urls = [
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/DigitalSignature.jpg',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/butterfly.pdf',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/pdftools-icon.png',
    'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/TestFileResources/OpenSans-Semibold.ttf'
]
appearance_file_paths = ['DigitalSignature.jpg', 'butterfly.pdf', 'pdftools-icon.png', 'OpenSans-Semibold.ttf']
for url, path in zip(appearance_file_urls, appearance_file_paths):
    prepare_file(url, path)
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/Invoice_field.pdf'
input_path = 'Invoice_field.pdf'
prepare_file(input_url, input_path)
output_path = 'Signed_Invoice_field.pdf'

In [ ]:
def add_appearance_signature_field(certificate_file: str, password: str, appearance_config_file: str, input_path: str, output_path: str):
    # Create a session to the built-in cryptographic provider
    with Provider() as session:
        # Create signature configuration from PFX (or P12) file
        with io.FileIO(certificate_file, 'rb') as pfx_str:
            signature = session.create_signature_from_certificate(pfx_str, password)

            # Open input document
            with io.FileIO(input_path, 'rb') as input_pdf_stream:
                with Document.open(input_pdf_stream) as input_pdf_document:
                    # Choose first signature field
                    for field in input_pdf_document.signature_fields:
                        if field:
                            signature.field_name = field.field_name
                            break

                    # Create stream for output file
                    with io.FileIO(output_path, 'wb+') as output_stream:
                        # Create appearance configuration from either XML or JSON file
                        with io.FileIO(appearance_config_file, 'rb') as appearance_config_stream:
                            if os.path.splitext(appearance_config_file)[1].lower() == ".xml":
                                signature.appearance = Appearance.create_from_xml(appearance_config_stream)
                            else:
                                signature.appearance = Appearance.create_from_json(appearance_config_stream)

                            signature.appearance.custom_text_variables["company"] = "Daily Planet"

                            # Sign the input document
                            signer = Signer()
                            signer.sign(input_pdf_document, signature, output_stream)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("INSERT-LICENSE-KEY")

    # Sign the input document
    add_appearance_signature_field(certificate_file, password, appearance_config_file, input_path, output_path)

    print(f"Successfully created file {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")